# **Saving CoinGecko Cryptocurrency Tickers to CSV**

## **Purpose of this Notebook**
The purpose of this notebook is to fetch, clean, and store cryptocurrency tickers and names from the CoinGecko API. The data is saved into a CSV file (`crypto_tickers.csv`) for efficient retrieval and future use in other machine learning or data analysis projects.

---

In [1]:
import requests
import pandas as pd
import time

In [2]:
def fetch_crypto_tickers():
    """Fetch the first 10 pages of cryptocurrency tickers from CoinGecko API with rate-limit handling."""
    url = "https://api.coingecko.com/api/v3/coins/markets"
    all_data = []
    max_pages = 10  # Fetch only the first 10 pages
    max_per_page = 250
    max_attempts = 3

    for page in range(1, max_pages + 1):
        params = {
            "vs_currency": "usd",
            "order": "market_cap_desc",
            "per_page": max_per_page,
            "page": page
        }

        for attempt in range(max_attempts):
            try:
                response = requests.get(url, params=params)
                response.raise_for_status()

                data = response.json()
                if not data:
                    break

                all_data.extend([{"ticker": f"{coin['symbol'].upper()}-USD", "name": coin['name']} for coin in data])
                print(f"Page {page} fetched successfully.")

                # Respect API rate limits
                time.sleep(1.5)  # Adjust sleep time if needed
                break

            except requests.exceptions.HTTPError as e:
                if response.status_code == 429:
                    print("Rate limit reached. Sleeping for 60 seconds...")
                    time.sleep(60)
                else:
                    print(f"Error fetching data on page {page}: {e}")
                    return pd.DataFrame(all_data)

    # Save results to CSV
    df = pd.DataFrame(all_data)
    output_file = "cryptoTickers.csv"
    df.to_csv(output_file, index=False)
    print(f"Data successfully written to '{output_file}'. Total tickers: {len(df)}")
    return df

# Fetch and display the first few rows
tickers_df = fetch_crypto_tickers()
display(tickers_df.head())

Page 1 fetched successfully.
Page 2 fetched successfully.
Page 3 fetched successfully.
Page 4 fetched successfully.
Page 5 fetched successfully.
Page 6 fetched successfully.
Page 7 fetched successfully.
Page 8 fetched successfully.
Page 9 fetched successfully.
Rate limit reached. Sleeping for 60 seconds...
Page 10 fetched successfully.
Data successfully written to 'cryptoTickers.csv'. Total tickers: 2500


,ticker,name
0,BTC-USD,Bitcoin
1,ETH-USD,Ethereum
2,USDT-USD,Tether
3,XRP-USD,XRP
4,SOL-USD,Solana


## **Why Save the Tickers to a CSV File?**

1. **Reduce API Calls:**  
   By saving the fetched tickers to a CSV file, we avoid overwhelming CoinGecko's servers with repeated requests. This is important because API providers often have strict rate limits.

2. **Efficient Data Handling:**  
   Loading data from a local file is faster than querying the API repeatedly, especially when performing multiple analyses or experiments.

3. **Data Consistency:**  
   By saving a snapshot of the tickers, we maintain consistency in the dataset. This ensures that our analyses aren't affected by dynamic changes in the data (e.g., new coins being added or removed).

4. **Reproducibility:**  
   Storing the data ensures that future experiments can be replicated under the same conditions, even if the API data changes over time.

5. **Data Pipeline Integration:**  
   The CSV format allows seamless integration with other tools in the data ecosystem, including pandas, visualization libraries, and machine learning models.

---

## **Next Steps**
1. Load the `cryptoTickers.csv` into your data CryptoPredictor.  
2. These tickers will be used to make the calls to yfinance when fetching an specific crypto.